<a href="https://colab.research.google.com/github/mahopman/IEMB-Net/blob/main/evidence_integration/generate_interval_pmcids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = 'Mia Hopman'

In [20]:
local_path = '/IEBM-Net_Data'
evidence_integration_path = f'{local_path}/evidence_integration'
clinical_trial_path = f'{local_path}/clinical_trials'

In [16]:
## extract NCTID from pubmed text and map to PMCID
import os
import glob
import json
import re

pmcid2nctid = {}

files = glob.glob(f'{evidence_integration_path}/evidence_inference/txt_files/PMC*.txt')

for file in files:
    pmcid = file.split('/')[-1].split('.')[0]

    with open(file) as f:
        text = f.read()

    matches = re.findall(r'NCT\d+', text)
    if matches:
        for match in matches:
            pmcid2nctid[pmcid] = match

with open(f'{evidence_integration_path}/pmcid2nctid.json', 'w') as f:
    json.dump(pmcid2nctid, f)

In [23]:
# map pmcid to intervention type
nctid2intervention = json.load(open(f'{clinical_trial_path}/nctid2intervention.json'))
pmcid2nctid = json.load(open(f'{evidence_integration_path}/pmcid2nctid.json'))

pmcid2intervention = {}

for pmcid, nctid in pmcid2nctid.items():
    try:
        intervention = nctid2intervention[nctid]
    except:
        continue
    pmcid2intervention[pmcid] = intervention

with open(f'{evidence_integration_path}/pmcid2intervention.json', 'w') as f:
    json.dump(pmcid2intervention, f)

In [25]:
# create list of pmids for each intervention type
pmcid2intervention = json.load(open(f'{evidence_integration_path}/pmcid2intervention.json'))

intervention2ids = {}

for pmcid, interventions in pmcid2intervention.items():
    for intervention in interventions:
        if intervention not in intervention2ids.keys():
            intervention2ids[intervention] = [pmcid]
        else:
            intervention2ids[intervention].append(pmcid)


with open(f'{evidence_integration_path}/intervention2ids.json', 'w') as f:
    json.dump(intervention2ids, f)

In [26]:
# check intervention distributions
for intervention in intervention2ids.keys():
    print(intervention, len(intervention2ids[intervention]))

DRUG 479
BEHAVIORAL 193
OTHER 208
DEVICE 96
DIETARY_SUPPLEMENT 76
RADIATION 17
PROCEDURE 85
BIOLOGICAL 56
GENETIC 1
